# Inspecting ModelSelectorResult
When we go down from multiple time-series to single time-series, the best way how to get access to all relevant information to use/access `ModelSelectorResult` objects

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [12, 6]

In [ ]:
from hcrystalball.model_selection import ModelSelector
from hcrystalball.utils import get_sales_data
from hcrystalball.wrappers import get_sklearn_wrapper
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df = get_sales_data(n_dates=365*2, 
                    n_assortments=1, 
                    n_states=1, 
                    n_stores=2)
df.head()

In [ ]:
# let's start simple
df_minimal = df[['Sales']]

In [ ]:
ms_minimal = ModelSelector(frequency='D', horizon=10)

In [ ]:
ms_minimal.create_gridsearch(
    n_splits=2,
    between_split_lag=None,
    sklearn_models=False,
    sklearn_models_optimize_for_horizon=False,
    autosarimax_models=False,
    prophet_models=False,
    tbats_models=False,
    exp_smooth_models=False,
    average_ensembles=False,
    stacking_ensembles=False)

In [ ]:
ms_minimal.add_model_to_gridsearch(get_sklearn_wrapper(LinearRegression, hcb_verbose=False))
ms_minimal.add_model_to_gridsearch(get_sklearn_wrapper(RandomForestRegressor, random_state=42, hcb_verbose=False))

In [ ]:
ms_minimal.select_model(df=df_minimal, target_col_name='Sales')

## Ways to access ModelSelectorResult

There are three ways how you can get to single time-series result level.

- First is over `.results[i]`, which is fast, but does not ensure, that results are loaded in the same order as when they were created (reason for that is hash used in the name of each result, that are later read in alphabetic order)
- Second and third uses `.get_result_for_partition()` through `dict` based partition 
- Forth does that using `partition_hash` (also in results file name if persisted)

In [ ]:
result = ms_minimal.results[0]
result = ms_minimal.get_result_for_partition({'no_partition_label': ''})
result = ms_minimal.get_result_for_partition(ms_minimal.partitions[0])
result = ms_minimal.get_result_for_partition('fb452abd91f5c3bcb8afa4162c6452c2')

## ModelSelectorResult is rich
As you can see below, we try to store all relevant information to enable easy access to data, that is otherwise very lenghty.

In [ ]:
result

### Traning data

In [ ]:
result.X_train

In [ ]:
result.y_train

### Data behind plots
Ready to be plotted or adjusted to your needs

In [ ]:
result.df_plot

In [ ]:
result.df_plot.tail(50).plot();

In [ ]:
result

## Best Model Metadata
That can help to filter for example `cv_data` or to get a glimpse on which parameters the best model has

In [ ]:
result.best_model_hash

In [ ]:
result.best_model_name

In [ ]:
result.best_model_repr

### CV Results
Get information about how our model behaved in cross validation

In [ ]:
result.best_model_cv_results['mean_fit_time']

Or how all the models behaved

In [ ]:
result.cv_results.sort_values('rank_test_score').head()

### CV Data
Access predictions made during cross validation with possible cv splits and true target values

In [ ]:
result.cv_data.head()

In [ ]:
result.cv_data.drop(['split'], axis=1).plot();

In [ ]:
result.best_model_cv_data.head()

In [ ]:
result.best_model_cv_data.plot();

## Plotting Functions
With `**plot_params` that you can pass depending on your plotting backend 

In [ ]:
result.plot_result(plot_from='2015-06', title='Performance', color=['blue','green']);

In [ ]:
result.plot_error(title='Error');

## Convenient Persist Method

In [ ]:
result.persist?